In [1]:
import keras
import cv2
import os
import gc
import numpy as np
import pandas as pd
import random as rand
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Flatten, Dense
from keras.utils.training_utils import multi_gpu_model


c:\users\vincenzo\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# We define in advance constants to build the model

INPUT_SHAPE = (96, 96, 3)
OUTPUT_SIZE = 2

LEARNING_RATE = 0.01
OPTIMIZER = keras.optimizers.Adam()
LOSS = 'binary_crossentropy'
METRIC = 'accuracy'

TRAIN_SIZE = 50121
EPOCHS = 10
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 50
BATCH_PER_EPOCH = int(TRAIN_SIZE/BATCH_SIZE) + 1

MODEL_PATH = "./binary_classifier/net_0_model.json"
WEIGHTS_PATH = "./binary_classifier/net_0_weights.h5"

In [3]:
def generator(epoch_tot, batch_size):

    sl_df = pd.read_csv('./sealions_train.csv', dtype={"coord_x": int, "coord_y": int, "class": str, "filename": str})
    bkg_df = pd.read_csv('./empty_train.csv', dtype={"coord_x": int, "coord_y": int, "class": str, "filename": str})
    for epoch_act in range(0, epoch_tot):
        training_df = training_df_gen(sl_df, bkg_df) 
        batch_count = 0
        patched_images = []
        for row in training_df.iterrows():       
            patches = []
            classes = []
            image_name = row[1]['filename']
            if image_name not in patched_images:    # open image only if not opened yet
                patched_images.append(image_name)
                image = cv2.imread("./kaggle_sea_lions/Train/" + image_name) 
            if row[1]['class'] == "background":
                patches.append(extract_background_patch(image, row[1]['coord_x'], row[1]['coord_y']))
                classes.append([0, 1])    
            else:
                patches.append(extract_sea_lion_patch(image, row[1]['coord_x'], row[1]['coord_y'], epoch_act, epoch_tot))
                classes.append([1, 0])
            batch_count += 1
            if batch_count >= batch_size:
                x = np.array(patches)
                y = np.array(classes)
                yield x, y

In [4]:
def training_df_gen(sea_lions_df, background_df):
    
    bkg_sub_df = background_df.sample(len(sea_lions_df))
    training_df = (bkg_sub_df.append(sea_lions_df)).sample(frac=1)  # shuffle
    training_df.sort_values(by="filename")
    return training_df

In [5]:
# Create data generator for augmented images

datagen = ImageDataGenerator(
    # Define rotation range between 0 and 360 degrees
    rotation_range=360,
    # Define range of random horizontal shifts (expressed as fraction of total width)
    width_shift_range=0.2,
    # Define range of random vertical shifts (expressed as fraction of total height)
    height_shift_range=0.2,
    # Define shear intensity
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

In [6]:
# Background patches extractor
def extract_background_patch(image, coord_x, coord_y):
    patch = image[coord_y-48:coord_y+48, coord_x-48:coord_x+48, :]
    return patch

In [7]:
# Sea lions patches extractor
def extract_sea_lion_patch(image, coord_x, coord_y, epoch, n_epochs):
    if coord_x < 72:
        coord_x = 0
    elif coord_x > len(image[0]) - 72:
        coord_x = len(image[0]) - 144
    else:
        coord_x = coord_x - 72
    if coord_y < 72:
        coord_y = 0
    elif coord_y > len(image) - 72:
        coord_y = len(image) - 144
    else:
        coord_y = coord_y - 72
    external_patch = image[coord_y:coord_y+144, coord_x:coord_x+144, :]
    if rand.uniform(0.0, 1.0) < (epoch/n_epochs):
        # Perform transformation
        for aug_img in datagen.flow(img_to_array(external_patch), [1, 0]):
            external_patch = array_to_img(aug_img)
            break
    patch = external_patch[72-48:72+48, 72-48:72+48, :]
    return patch

In [8]:
sl_val_df = pd.read_csv('./sealions_validation.csv', dtype={"coord_x": int, "coord_y": int, "class": str, "filename": str})
bkg_val_df = pd.read_csv('./empty_validation.csv', dtype={"coord_x": int, "coord_y": int, "class": str, "filename": str})
val_df = pd.concat([sl_val_df, bkg_val_df])
file_names = val_df.filename.unique()
val_patches = []
val_classes = []
i = 0
for file in file_names:
    image = cv2.imread("./kaggle_sea_lions/Train/" + file)
    df = val_df[val_df['filename'] == file]
    for row in df.iterrows():
        x = row[1]['coord_x']
        y = row[1]['coord_y']
        if x < 48:
            x = 0
        elif x > len(image[0]) - 48:
            x = len(image[0]) - 96
        else:
            x = x - 48
        if y < 48:
            y = 0
        elif y > len(image) - 48:
            y = len(image) - 96
        else:
            y = y - 48
        patch = image[y:y+96, x:x+96, :]
        val_patches.append(patch)
        if row[1]['class'] == "background":
            val_classes.append([0, 1])
        else:
            val_classes.append(([1, 0]))
    image = []
    gc.collect()
    i += 1
    if i > 10:
        break
X_val = np.array(val_patches)
Y_val = np.array(val_classes)

In [9]:
# Build parallel model (multi gpu)

model = Sequential()
# First layer
model.add(Convolution2D(8, (5, 5), activation='relu', padding='valid', input_shape=INPUT_SHAPE))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# Second layer
model.add(Convolution2D(5, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# Third layer
model.add(Convolution2D(5, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# Fourth layer
model.add(Convolution2D(10, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Flatten())

# model.add(Dropout(0.5))
model.add(Dense(OUTPUT_SIZE, activation='softmax'))

parallel_model = multi_gpu_model(model, gpus=2)
parallel_model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=[METRIC])

In [10]:
# Train (multi gpu)

# Fit model on training data
history = parallel_model.fit_generator(generator(EPOCHS, BATCH_SIZE), steps_per_epoch=BATCH_PER_EPOCH, epochs=EPOCHS,
                                       verbose=1, validation_data=(X_val, Y_val))

Epoch 1/10
